# Examples 6.2, 6.3: Logical Operators
This notebook illustrates the operation of the Logical Operator algorithm for Code 1 (Example 6.2). 

To calculate logical operators for Code 2 (Example 6.3), un-comment the code where indicated.

## Diagonal Logical Operator Generators $\mathbf{L}_Z$

The algorithm assumes we are given the Codewords as input. 

We form $E_L$ from the Z-support of the code words, $E$, and appending the code word index $\mathbf{i}$ to each row. 

We then calculate the Howell basis $K$ of the kernel of $E_L$.

To run different scenarios click here: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/m-webster/XPFpackage/HEAD?urlpath=tree/Examples)

This code is Copyright 2021-22 Mark Webster, and is made available subject to [GPL licensing](https://www.gnu.org/licenses/gpl-3.0.en.html).

In [1]:
import add_parent_dir
import numpy as np
from common import *
from NSpace import *
from XPAlgebra import *
from XPCodes import *

## Code 1
genstr = 'XP_8(8|0000000|6554444),XP_8(7|1111111|1241234),XP_8(1|1110000|3134444)'

## Code 2
## Un-comment to calculate logical operators for Code 2 (Example 6.3)
# genstr = 'XP8(0|0000000|1322224),XP8(12|1111111|1234567)'

setVerbose(False)
G, N = str2XP(genstr)
n = XPn(G)
print('The Code Generators are G:')
print(XP2Str(G,N),"\n")
C = Code(G,N)
Codewords = getVal(C,'Codewords')
print('The code words are:')
CW_index = []
d = len(Codewords)
for i in range(d):
    print(f'|k_{i}> = {State2Str(Codewords[i],N)}')
    cwi = set2Bin(d,[i])
    r = len(Codewords[i])
    CW_index.append([list(cwi)]*r)
print('\n')
E = np.vstack([XPx(c) for c in Codewords])
print('E_L is the ZSupport of the code words, E, plus the code word index:')
CW_index = np.vstack(CW_index)
EL = np.hstack([E,CW_index])
print(ZmatPrint(EL,2))
print('\n')
nsp = NSpace(EL,N)
nsp.simplifyKer()
K = nsp.K
print('K is the Howell basis of Ker(E_L):')
print(ZmatPrint(K,N))
print(f'First {n} columns of K represent Z components of diagonal logical operators: LZz')
LZz = K[:,:n]
print(ZmatPrint(LZz,N))
LI = getVal(C,'LI')
MX,MZ = splitDiag(LI)
MZz = XPz(MZ)
print('Z components of diagonal logical identites are: MZz=')
print(ZmatPrint(MZz,N))
print('We take the residue of each row of LZz with respect to MZz: Res_ZN(MZz,LZz)=')
LZz = [matResidual(MZz,z,N)[0] for z in LZz]
print(ZmatPrint(LZz,N))
print('Calculate Howell basis of the residue and remove zero rows. The diagonal logical operator generators LZ are then: ')
nspLZ = NSpace(LZz,N)
LZz = nspLZ.H
LZ = makeXP(0,0,LZz)
print(XP2Str(LZ,N))

The Code Generators are G:
XP_8( 8|0000000|6554444)
XP_8( 7|1111111|1241234)
XP_8( 1|1110000|3134444) 

The code words are:
|k_0> = |0000001>+w6/16|0001110>+w9/16|1110001>+w15/16|1111110>
|k_1> = |0000010>+w4/16|0001101>+w9/16|1110010>+w13/16|1111101>
|k_2> = |0000100>+w2/16|0001011>+w9/16|1110100>+w11/16|1111011>
|k_3> = |0000111>+|0001000>+w9/16|1110111>+w9/16|1111000>


E_L is the ZSupport of the code words, E, plus the code word index:
00000011000
11100011000
00011101000
11111101000
00000100100
11100100100
00011010100
11111010100
00001000010
11101000010
00010110010
11110110010
00001110001
11101110001
00010000001
11110000001


K is the Howell basis of Ker(E_L):
10700000000
01700000000
00022262666
00004044040
00000444400
First 7 columns of K represent Z components of diagonal logical operators: LZz
1070000
0170000
0002226
0000404
0000044
Z components of diagonal logical identites are: MZz=
1070000
0170000
0004444
We take the residue of each row of LZz with respect to MZz: Res_ZN(MZz,

## Non-Diagonal Logical Operator Generators $\mathbf{L}_X$
The process to find the non-diagonal generators is as follows:
1. Find the X components of the generators
2. Find the phase and Z components which are consistent with the relative phases between computational basis elements in code words


### Step 1: X-Components: 
Firstly, we find the X-components of the generators.  Let $E_m$ be the orbit representatives. Take any element $\mathbf{e}_0 \in E_m$ and let $\mathbf{e}_0 + E_m = \{\mathbf{e}_0 \oplus \mathbf{e}  : \mathbf{e} \in E_m\}$. 

The group of transitions between elements of $E_m$ is given by $T = \{\mathbf{x} \in \mathbf{e}_0 + E_m :  \mathbf{x} \oplus \mathbf{e} \in E_m, \forall \mathbf{e} \in E_m\}$. 

The X components which generate the transitions $T$ are given by $L_X = \text{RREF}_{\mathbb{Z}_2}(T)$.

In the function cosetDecomposition(), a more efficient algorithm is used but the below illustrates the general method:

In [2]:

Em = getVal(C,'Em')
print('Em=')
EmStr = ZMat2str(Em)
print("\n".join(EmStr))
e0 = Em[0]
print('\ne0 =',EmStr[0])
Em0 = np.mod(e0 + Em,2)
print('\nEm + e0 =')
print(ZmatPrint(Em0,2))
startTimer()
T = []
EmSet = {tuple(m) for m in Em}
for x in Em0:
    mux = np.mod(Em + x,2)
    if np.all([tuple(m) in EmSet for m in mux]):
        T.append(x)
T = ZMat(T)

print('\nT =')
print(ZmatPrint(T,2))
nspLX = NSpace(T,2)
LXx = nspLX.H
print('\nLXx = RREF(T) =')
print(ZmatPrint(LXx,2))
print('\nNaive Method: Elapsed Time',"{:e}".format(elapsedTime()))
Eq,LXx = cosetDecomposition(Em)
print('\ncosetDecomposition Function: Elapsed Time',"{:e}".format(elapsedTime()))
print('LXx =')
print(ZmatPrint(LXx,2))
print('Eq =')
print(ZmatPrint(Eq,2))

Em=
0000001
0000010
0000100
0000111

e0 = 0000001

Em + e0 =
0000000
0000011
0000101
0000110

T =
0000000
0000011
0000101
0000110

LXx = RREF(T) =
0000101
0000011

Naive Method: Elapsed Time 1.562500e-02

cosetDecomposition Function: Elapsed Time 0.000000e+00
LXx =
0000101
0000011
Eq =
0000001




### Step 2: Phase and Z-Components: 
Assume that the codewords $|\kappa_i\rangle$ are written in orbit form. The algorithm for finding the phase and Z-components of the operators for a given $\mathbf{x}$ is as follows:

- Let $\mathbf{e}'_{ij} = \mathbf{e}_{ij} \oplus \mathbf{x}$, and let $p'_{ij}$ be the phase of $\mathbf{e}'_{ij}$in the code words
- Let $p''_{ij} = (p'_{ij} - p_{ij}) \mod 2N$ - this is the change in phase when applying the operator $XP_N(0|\mathbf{x}|\mathbf{0})$
- For there to be a valid solution, for fixed $i$ the $p''_{ij}$ are all either even or odd. Let $a_i = p''_{i0} \mod 2$ be an adjustment factor
- Let $p'''_{ij} = (p''_{ij} - a_i)/2$ so that $p'''_{ij} \in \mathbb{Z}_N$, and let $\mathbf{p}'''$ be the binary vector with the $p'''_{ij}$ as components
- Find a solution $\mathbf{b} = (\mathbf{z}|\mathbf{q}) \in \mathbb{Z}_N^{n}\times \mathbb{Z}_N^{\dim(\mathcal{C})}$ such that $E_L \mathbf{b} \mod N = \mathbf{p}'''$ using linear algebra modulo $N$  (see Section \ref{sec:linalgmodN})
- Let $A = XP_N(a_i|\mathbf{x}|\mathbf{z})$. For $A$ to be a valid X operator, then applying $A^2$ to a basis vector $|\mathbf{e}\rangle$ where $\mathbf{e} \in E$ should result in there being no net phase being applied ie $A^2|\mathbf{e}\rangle = |\mathbf{e}\rangle$. Hence we require $A^2 \in \langle \mathbf{M}_Z \rangle$. Adjust the phase and Z component to ensure this using the intersection of affine spans technique in Appendix A.

In [3]:
# M,Em,Eq,LXx = getVals(C,['LI','Em','Eq','LXx'])
LO = getVal(C,'LO')
CWL = C.getCWL(N)
nsp = getNsp(CWL,N)
setVerbose(True) 
print('\nFirst, we find Z components for each x in LXx such that XP_N(0|x|z) is a logical operator:')
LX = CW2LX(CWL,LXx,nsp,N)
print("\nList of non-diagonal logical operators:")
print(XP2Str(LX,N))
print('\nNext, we adjust the phase and Z components to ensure that the square of operator is in <MZ>:')
LX = getVal(C,'LogicalX')
print("\nList of logical X operators:")
print(XP2Str(LX,N))
setVerbose(False) 


First, we find Z components for each x in LXx such that XP_N(0|x|z) is a logical operator:
p''' represents the change in phase to each basis element in the codewords when applying the operator  XP_8( 0|0000101|0000000)
[4 4 2 2 4 4 6 6 4 4 2 2 4 4 6 6]
b = (z|q) represents the Z component and phase vector of an operator which applies the change in phase p''':
[0 0 0 4 6 0 4 0 6 4 2]
p''' represents the change in phase to each basis element in the codewords when applying the operator  XP_8( 0|0000011|0000000)
[2 2 1 1 2 2 1 1 6 6 7 7 6 6 7 7]
b = (z|q) represents the Z component and phase vector of an operator which applies the change in phase p''':
[0 0 0 6 2 5 6 4 0 1 1]

List of non-diagonal logical operators:
XP_8( 0|0000101|0004604)
XP_8( 0|0000011|0006256)

Next, we adjust the phase and Z components to ensure that the square of operator is in <MZ>:

List of logical X operators:
XP_8( 2|0000101|0000204)
XP_8( 1|0000011|0000034)


The Logical XP Operator group $\mathcal{L}_\text{XP} = \langle \omega I, \mathbf{M},\mathbf{L}\rangle$ where:

In [4]:
print('Operator applying w to each codeword: wI =')
wI = getVal(C,'wI')
print(XP2Str(wI,N))
print('\nLogical Identity Generators: M =')
print(XP2Str(LI,N))
print('\nNon-trivial Logical Operator Generators: L =')
LO = np.vstack([LX,LZ])
print(XP2Str(LO,N))

print('\nCheck Logical operators by testing if group commutators with elements of M are in <MZ>:')
for L in LO:
    print(XP2Str(L,N), isLO(L,LI,N))

Operator applying w to each codeword: wI =
XP_8( 1|0000000|0000000)

Logical Identity Generators: M =
XP_8( 9|1110000|0070000)
XP_8(14|0001111|0001234)
XP_8( 0|0000000|1070000)
XP_8( 0|0000000|0170000)
XP_8( 8|0000000|0004444)

Non-trivial Logical Operator Generators: L =
XP_8( 2|0000101|0000204)
XP_8( 1|0000011|0000034)
XP_8( 0|0000000|0002226)
XP_8( 0|0000000|0000404)
XP_8( 0|0000000|0000044)

Check Logical operators by testing if group commutators with elements of M are in <MZ>:
XP_8( 2|0000101|0000204) True
XP_8( 1|0000011|0000034) True
XP_8( 0|0000000|0002226) True
XP_8( 0|0000000|0000404) True
XP_8( 0|0000000|0000044) True
